# 검증 세트(validation set)

테스트 세트를 사용하지 않고 훈련 세트를 다시 나눠서 훈련하는 방식.

전체 데이터의 20%를 테스트 세트로, 나머지 80%의 훈련 세트 중 20%를 검증 세트로 나눈다.(이러한 과정은 문제에 따라 다르다. 훈련 데이터가 많다면 일부 데이터만 떼어내도 전체 데이터를 대표하는 데 문제가 없다.)


In [1]:
import pandas as pd

wine = pd.read_csv('https://bit.ly/wine_csv_data') 

In [2]:
wine.head()

,alcohol,sugar,pH,class
0,9.4,1.9,3.51,0.0
1,9.8,2.6,3.20,0.0
2,9.8,2.3,3.26,0.0
3,9.8,1.9,3.16,0.0
4,9.4,1.9,3.51,0.0


In [3]:
wine.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6497 entries, 0 to 6496
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   alcohol  6497 non-null   float64
 1   sugar    6497 non-null   float64
 2   pH       6497 non-null   float64
 3   class    6497 non-null   float64
dtypes: float64(4)
memory usage: 203.2 KB


In [4]:
data = wine[['alcohol', 'sugar', 'pH']].to_numpy()
target = wine['class'].to_numpy()

In [5]:
from sklearn.model_selection import train_test_split

train_input, test_input, train_target, test_target = train_test_split(
    data, target, test_size=0.2, random_state=42
)

In [6]:
print(train_input.shape, test_input.shape)

(5197, 3) (1300, 3)


In [7]:
sub_input, val_input, sub_target, val_target = train_test_split(                   # 훈련 세트의 20%를 검증 데이터로 만든다.
    train_input, train_target, test_size=0.2, random_state=42
)

In [8]:
print(sub_input.shape, val_input.shape, test_input.shape)

(4157, 3) (1040, 3) (1300, 3)


### DecisionTreeClassifier 사용

훈련, 검증, 테스트 세 가지 세트로 나눈 데이터를 결정 트리 알고리즘(decision tree : DecisionTreeClassifier)을 사용하여 score를 확인한다.

In [9]:
from sklearn.tree import DecisionTreeClassifier

dt = DecisionTreeClassifier()

dt.fit(sub_input, sub_target)                                                      # 훈련을 통해 검증 세트의 score를 확인

print(dt.score(sub_input, sub_target))
print(dt.score(val_input, val_target))                                             # 트리 알고리즘을 사용했으므로 과대적합 현상의 발생이 쉬워진다.

0.9971133028626413
0.8615384615384616


훈련 세트에 과대적합 된 모습을 확인할 수 있다. 즉, 매개변수를 바꿔서 더 좋은 모델을 찾아야 한다.

## 교차 검증(cross validation)

검증 세트를 떼어 내어 평가하는 과정을 여러 번 반복한다.(3-폴드 교차 검증, 보통은 5-폴드나 10-폴드를 사용한다.)

전체 데이터의 80%인 훈련 세트를 3등분으로 나눠 1, 2번을 훈련 세트, 3번을 검증 세트로 하여 훈련을 하고, 다음으로 1, 3번을 훈련 세트, 2번을 검증 세트로 하여 훈련을 진행, 마지막으로 2, 3번을 훈련 세트, 1번을 검증 세트로 하여 훈련한다.

In [10]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


이렇게 하면 데이터의 80~90%까지를 훈련에 사용할 수 있다.(검증 세트가 줄어들지만 각 폴드에서 계산한 검증 점수를 평균하기 때문에 안정된 점수로 생각할 수 있다.)

### cross_validate() 메소드

사이킷런에는 **cross_validate**라는 교차 검증 함수가 있다.

사용법 : 먼저 평가할 모델 객체를 첫 번째 매개변수로 지정 -> **직접 검증 세트를 떼어내지 않고 훈련 세트 전체를 cross_validate() 함수에 전달**.

cross_validate()을 사용하면 **딕셔너리의 형태**로 반환되며 각각의 키에 값이 저장된다.
- fit_time : 모델을 훈련하는 시간
- score_time : 모델을 검증하는 시간
- **test_score** : 5개(cv 매개변수 디폴트값)의 score.(**모든 score를 평균하면 교차 검증의 최종 점수를 알 수 있다**.)

`사이킷런에는 cross_validate() 함수의 전신인 cross_val_score() 함수도 있다.`
`이 함수는 cross_validate() 함수의 결과 중에서 test_score 값만 반환한다.`

In [11]:
from sklearn.model_selection import cross_validate                                 # sub_input, sub_target을 만들지 않아도 모든 과정을 진행해준다.

scores = cross_validate(dt, train_input, train_target)                             # cross_validate의 디폴트값은 5-폴드 교차 검증이다.

print(scores)                                                                      # cross_validate는 딕셔너리 형태로 반환된다.

{'fit_time': array([0.01449943, 0.00904226, 0.00790858, 0.00775552, 0.00762939]), 'score_time': array([0.00178766, 0.00105858, 0.00183892, 0.00207829, 0.00106645]), 'test_score': array([0.8625    , 0.84423077, 0.87584216, 0.85370549, 0.84023099])}


In [12]:
import numpy as np                                                                 #                              ^
                                                                                   #                              |
print(np.mean(scores['test_score']))                                               # 교차 검증의 최종 점수는 test_score 키에 담긴 5개의 점수를 평균하여 얻을 수 있다.

0.8553018805064042


cross_validate()는 훈련 세트를 **섞어서 폴드로 나누지 않는다**.(대신 train_test_split을 사용했기 때문에 따로 섞을 필요가 없었다.)

### cross_validate(cv=StratifiedKFold) 사용하여 섞은 후 폴드하기

교차 검증을 할 때 훈련 세트를 섞으려면 **분할기(splitter)**를 지정.

cross_validate() 함수는 기본적으로 **분류 모델일 경우에는 StratifiedKFold**를, **회귀 모델일 경우 KFold**를 사용한다.

In [13]:
from sklearn.model_selection import StratifiedKFold                                # 레드 와인과 화이트 와인을 나누는 분류 모델이므로 StratifiedKFold를 사용
scores = cross_validate(dt, train_input, train_target, cv=StratifiedKFold())       # cv=StratifiedKFold()를 사용하여 섞으면서 5등분으로 나눠 교차 검증을 하라는 말이 된다.
print(np.mean(scores['test_score']))

0.8568399718664397


#### 10-폴드 교차 검증

훈련 세트를 섞은 후, 10-폴드 교차 검증을 수행하려면 다음과 같다.

In [14]:
splitter = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
scores = cross_validate(dt, train_input, train_target, cv=splitter)
print(np.mean(scores['test_score']))

0.8576137542611532


회귀에서 사용하는 KFold도 동일한 방식으로 사용할 수 있다.

# 하이퍼파라미터 튜닝

결정 트리의 매개변수 값을 바꿔가며 **가장 좋은 성능이 나오는 모델 찾기**(테스트 세트를 사용하지 않고 **교차 검증을 이용**.)



머신러닝 모델이 학습하는 파라미터를 모델 파라미터라고 부른다.

반면, 모델이 학습할 수 없어서 사용자가 지정해야만 하는 파라미터를 하이퍼파라미터라고 한다.

이런 하이퍼파라미터는 모두 클래스나 메서드의 매개변수로 표현된다.

`사람의 개입이 없어 하이퍼파라미터 튜닝을 자동으로 수행하는 기술을 'AutoML'이라고 부른다.`


## 그리드 서치(grid search) - GridSearchCV

하나의 하이퍼파라미터의 최적의 값을 찾은 후, 그 값을 고정시키고 다른 하이퍼파라미터의 값을 변경하면 그 값이 바뀌게 된다. 즉, **두 매개변수를 동시에 바꿔가며 최적의 값을 찾아야 한다는 말이다.**

이러한 문제는 쉽게 해결해주는 방식이 **GridSearchCV**이다.

- **GridSearchCV는**  하이퍼파라미터 탐색과 교차 검증을 한 번에 수행한다. 다시 말해 **cross_validate() 함수를 호출하지 않아도 된다.**

- GridSearchCV는 클래스를 임포트하고 탐색할 매개변수와 탐색할 값을 리스트를 **딕셔너리**로 만든다.(키는 **'min_impurity', 'max_depth', 'min_samples_split'**)

- GridSearchCV는 클래스에 탐색할 대상과 **params** 변수를 전달하여 그리드 서치 객체를 만든다. 

- GridSearchCV 클래스의 **cv** 매개변수는 5(**5-폴드 교차 검증**)이다.

- GridSearchCV 클래스의 **n_jobs** 매개변수는 **병렬 실행에 사용할 CPU 코어 수**를 지정하는 것으로 '모든 것'을 뜻하는 '-1'을 주로 쓴다.

In [15]:
from sklearn.model_selection import GridSearchCV
params = {'min_impurity_decrease' : [0.0001, 0.0002, 0.0003, 0.0004, 0.0005]}

In [16]:
gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1)      # DecisionTreeClassifier - 결정 트리 

gs.fit(train_input, train_target)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'min_impurity_decrease': [0.0001, 0.0002, 0.0003,
                                                   0.0004, 0.0005]})

In [17]:
dt = gs.best_estimator_                                                            # 모델 중에서 검증 점수가 가장 높은 모델의 매개변수 조합으로 전체 훈련 세트에서 자동으로 다시 모델을 훈련한다.
print(dt.score(train_input, train_target))                                         # 이 모델은 gs객체의 best_estimator_에 저장되어있다.

0.9615162593804117


In [18]:
print(gs.best_params_)                                                             # 그리드 서치로 찾은 최적의 매개변수는 best_params_ 속성에 저장되어있다.

{'min_impurity_decrease': 0.0001}


In [19]:
print(gs.cv_results_['mean_test_score'])                                            # 각 매개변수에서 수행한 교차 검증의 평균 점수는 cv_results 속성의 'mean_test_score'에 저장되어있다.

[0.86819297 0.86453617 0.86492226 0.86780891 0.86761605]


- **best_estimator_**는 검증 점수가 가장 높은 모델을 가지고 다시 **훈련**을 하는 것. 다시 말해, **score가 저장**되어있다.

- **best_params_**는 검증 점수가 가장 높은 모델이 나왔을 때, **사용했던 최적의 매개변수**가 저장되어있다.

- **cv_results_ 속성의 'mean_test_score'**에는 각 매개변수에서 수행한 **교차 검증의 평균 점수**가 저장되어있다.

In [20]:
best_index = np.argmax(gs.cv_results_['mean_test_score'])                          # numpy 함수, argmax() 를 이용해서도 가장 큰 값의 인덱스를 추출할 수 있다.
print(gs.cv_results_['params'][best_index])

{'min_impurity_decrease': 0.0001}


### 그리드 서치 과정 정리

1. 탐색할 **매개변수를 지정**

2. 그다음 훈련 세트에서 그리드 서치를 수행하여 **최상의 평균 검증 점수가 나오는 매개변수** 조합을 찾아 그리드 서치 객체(**best_params_**)에 저장.

3. **최상의 매개변수에서** (교차 검증에서 사용한 매개변수가 아닌) 전체 훈련 세트를 사용해 **최종 모델을 훈련**하여 그리드 서치 객체(**best_estimator_**)에 저장.

In [21]:
params = {'min_impurity_decrease' : np.arange(0.0001, 0.001, 0.0001),
          'max_depth' : range(5, 20, 1),
          'min_samples_split' : range(2, 100, 10)
          }

- **'min_impurity_decrease'** : 노드를 분할하기 위한 **불순도 감소 최소량**을 지정.

- **'max_depth'** : 트리의 **깊이를 제한.**

- **'min_samples_split'** : 노드를 나누기 위한 **최소 샘플 수** 지정.

In [ ]:
gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1)
gs.fit(train_input, train_target)

In [ ]:
print(gs.best_params_)                                                             # 어떤 값이 최선인가?

In [ ]:
dt = gs.best_estimator_                                                            # 최선인 값을 대입했을 때 확률은 얼마인가?
print(dt.score(train_input, train_target))

**그리드 서치(grid search)는** 매개변수를 일일이 바꿔가며 교차 검증을 수행하지 않고 원하는 **매개변수 값을 나열하여 자동으로 교차 검증을 수행**하여 최상의 매개변수를 찾을 수 있다.

________

## 랜덤 서치(random search) - RandomizedSearchCV

매개변수 값이 수치일 때 값의 범위나 간격을 미리 정하기 어려울 수 있다. 또, **너무 많은 매개변수 조건(값이 수치형이고 특히 연속적인 실숫값)**이 있어 그리드 서치 수행 시간이 오래 걸리 수 있다. 이럴 때 랜덤 서치를 사용.

매개변수 값의 목록을 전달하는 것이 아니라 **매개변수를 샘플링할 수 있는 확률 분포 객체를 전달**한다.

- **RandomizedSearchCV는**  하이퍼파라미터 탐색과 교차 검증을 한 번에 수행한다. 다시 말해 **cross_validate() 함수를 호출하지 않아도 된다.**

- RandomizedSearchCV는 클래스를 임포트하고 탐색할 매개변수와 탐색할 값을 리스트를 **딕셔너리**로 만든다.(키는 **'min_impurity', 'max_depth', 'min_samples_split', 'n_iter'**)

- RandomizedSearchCV는 클래스에 탐색할 대상과 **params** 변수를 전달하여 랜덤 서치 객체를 만든다. 

- RandomizedSearchCV 클래스의 **cv** 매개변수는 5(**5-폴드 교차 검증**)이다.

- RandomizedSearchCV 클래스의 **n_jobs** 매개변수는 **병렬 실행에 사용할 CPU 코어 수**를 지정하는 것으로 '모든 것'을 뜻하는 '-1'을 주로 쓴다.

- RandomizedSearchCV 클래스의 **n_iter=횟수** 매개변수는 정의된 범위에서 지정된 횟수만큼 **샘플링하여 교차 검증을 수행**하고 최적의 매개변수 조합을 찾는다.

### 싸이파이

`과학 라이브러리. 적분, 보간, 선형 대수, 확률 등을 포함한 수치계산 전용 라이브러리.`

#### uniform, randint - (중복 허용됨)

uniform, randint 클래스는 주어진 범위에서 고르게 값을 뽑는다.(**= 균등 분포에서 샘플링한다.**)
- uniform : 실수값을 뽑는다.

- randint : 정수값을 뽑는다.

In [ ]:
from scipy.stats import uniform, randint

rgen = randint(0, 10)
rgen.rvs(10)                                                                      # 범위 안에서 10개를 뽑아주세요.(중복 허용)

In [ ]:
np.unique(rgen.rvs(1000), return_counts=True)

In [ ]:
ugen = uniform(0, 1)
ugen.rvs(10)

In [ ]:
params = {
    'min_impurity_decrease' : uniform(0.0001, 0.001),
    'max_depth' : randint(20, 50),
    'min_samples_split' : randint(2, 25),
    'min_samples_leaf' : randint(1, 25)
}

- **'min_impurity_decrease'** : 노드를 분할하기 위한 **불순도 감소 최소량**을 지정.

- **'max_depth'** : 트리의 **깊이를 제한.**

- **'min_samples_split'** : 노드를 나누기 위한 **최소 샘플 수** 지정.

- **'min_samples_leaf'** : 리프 노드가 되기 위한 **최소 샘플의 개수**. 어떤 노드가 분할하여 만들어질 **자식 노드의 샘플 수가 이 값보다 작을 경우 분할하지 않는다.**

In [ ]:
from sklearn.model_selection import RandomizedSearchCV

gs = RandomizedSearchCV(DecisionTreeClassifier(random_state=42), params,
                        n_iter=100, n_jobs=-1, random_state=42)
gs.fit(train_input, train_target)

In [ ]:
print(gs.best_params_)

In [ ]:
print(np.max(gs.cv_results_['mean_test_score']))

In [ ]:
dt = gs.best_estimator_
print(dt.score(test_input, test_target))